# 준비 단계

data 파일을 받아서 data 폴더 안에 넣어둠

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import torch
from torch.jit import script, trace
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import csv
import random
import re
import os
import unicodedata
import codecs
from io import open
import itertools
import math

USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda' if USE_CUDA else 'cpu')

# 데이터 전처리

[코넬 영화-대사 말뭉치](https://www.cs.cornell.edu/~cristian/Cornell_Movie-Dialogs_Corpus.html)는 

- 영화 등장인물 10,292쌍 사이의 220,579 대화와
- 617개 영화로부터 9,035명의 인물과
- 총 304,713개의 발화로 이루어짐

이 데이터셋은 크고 분위기, 시기, 예의 정도에서 다양하며 이로 인해 우리 모델이 강인하길 기대할 수 있다.

먼저 데이터셋 초반부를 살펴보자.

In [2]:
corpus_name = 'Movie-Dialogs_Corpus'
corpus = os.path.join('data', corpus_name)

def printLines(file, n=10):
    with open(file, 'rb') as datafile:
        lines = datafile.readlines()
    for line in lines[:n]:
        print(line)
        
printLines(os.path.join(corpus, 'movie_lines.txt'))    

b'L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!\n'
b'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!\n'
b'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.\n'
b'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?\n'
b"L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.\n"
b'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow\n'
b"L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.\n"
b'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No\n'
b'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?\n'
b'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?\n'


# 형식 갖춘 데이터 파일 생성

편의를 위해 데이터를 탭으로 분리된 발화문과 응답문의 쌍으로 만들고자 한다.

다음 함수들을 사용하여 movie_lines.txt 파일을 분리할 것이다.

- `loadLines` : 각각의 라인을 다음과 같은 딕셔너리로 분리함 (라인ID, 등장인물ID, 영화ID, 등장인물, 대사)
- `loadConversations` : `loadLines` 로 분리한 라인들을 movie_conversations.txt 에 기초하여 그룹지음
- `extractSentencePairs` : 대화로부터 발화쌍을 추출함

In [3]:
# 각각의 라인을 딕셔너리로 분리함
def loadLines(fileName, fields):
    lines = {}
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # 필드 추출
            lineObj = {}
            for i, field in enumerate(fields):
                lineObj[field] = values[i]
            lines[lineObj['lineID']]= lineObj # 딕셔너리의 key를 lineID 로 지정
    return lines

# moive_conversations.txt(대사 목록) 를 토대로 'loadLines'의 라인들의 field를 그루핑
def loadConversations(fileName, lines, fields):
    conversations = []
    with open(fileName, 'r', encoding='iso-8859-1') as f:
        for line in f:
            values = line.split(" +++$+++ ")
            # 필드 추출
            convObj = {}
            for i, field in enumerate(fields):
                convObj[field] = values[i]
            # 문자열을 리스트로 변환 (convObj['utteranceIDs'] == "['L598485',...]")
            utterance_id_pattern = re.compile('L[0-9]+') # 발화ID를 하나씩 추출
            lineIds = utterance_id_pattern.findall(convObj['utteranceIDs'])
            # 라인들을 재정렬
            convObj['lines'] = []
            for lineId in lineIds:
                convObj['lines'].append(lines[lineId]) # 각 대사별로 lines 데이터 불러옴
            conversations.append(convObj)
    return conversations

# 대화들로부터 발화쌍을 추출
def extractSentencePairs(conversations):
    qa_pairs = []
    for conversation in conversations:
        # 대화의 모든 라인을 순환
        for i in range(len(conversation['lines'])-1): # 마지막줄은 대답이 없으므로 무시
            inputLine = conversation['lines'][i]['text'].strip()
            targetLine = conversation['lines'][i+1]['text'].strip()
            # 둘중 하나가 빈 샘플은 제외
            if inputLine and targetLine:
                qa_pairs.append([inputLine, targetLine])
    return qa_pairs
        


이제 이 함수들을 이용하여 formatted_movie_lines.txt 파일을 생성하자

In [4]:
# 새로운 파일의 경로 지정
datafile = os.path.join(corpus, 'formatted_movie_lines.txt')

delimiter = '\t'
delimiter = str(codecs.decode(delimiter, 'unicode_escape'))

# 대사 딕셔너리, 대화 리스트, 필드id 초기화
lines = {}
conversations = []
MOVIE_LINES_FIELDS = ["lineID", "characterID", "movieID", "character", "text"]
MOVIE_CONVERSATIONS_FIELDS = ["character1ID", "character2ID", "movieID", "utteranceIDs"]

# 대사들을 불러온 후 처리
print('\nProcessing corpus...')
lines = loadLines(os.path.join(corpus, 'movie_lines.txt'), MOVIE_LINES_FIELDS)
print('\nLoading conversations...')
conversations = loadConversations(os.path.join(corpus, 'movie_conversations.txt'),
                                 lines, MOVIE_CONVERSATIONS_FIELDS)

# 새로운 csv 파일을 저장
print('\nWriting newly formatted file...')
with open(datafile, 'w', encoding='utf-8') as outputfile:
    writer = csv.writer(outputfile, delimiter=delimiter, lineterminator='\n')
    for pair in extractSentencePairs(conversations):
        writer.writerow(pair)
        
# 샘플 라인 출력
print('\nSample lines form file:')
printLines(datafile)


Processing corpus...

Loading conversations...

Writing newly formatted file...

Sample lines form file:
b"Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\tWell, I thought we'd start with pronunciation, if that's okay with you.\r\n"
b"Well, I thought we'd start with pronunciation, if that's okay with you.\tNot the hacking and gagging and spitting part.  Please.\r\n"
b"Not the hacking and gagging and spitting part.  Please.\tOkay... then how 'bout we try out some French cuisine.  Saturday?  Night?\r\n"
b"You're asking me out.  That's so cute. What's your name again?\tForget it.\r\n"
b"No, no, it's my fault -- we didn't have a proper introduction ---\tCameron.\r\n"
b"Cameron.\tThe thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.\r\n"
b"The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister. 

In [5]:
lines['L1045']

{'lineID': 'L1045',
 'characterID': 'u0',
 'movieID': 'm0',
 'character': 'BIANCA',
 'text': 'They do not!\n'}

In [6]:
conversations[0]

{'character1ID': 'u0',
 'character2ID': 'u2',
 'movieID': 'm0',
 'utteranceIDs': "['L194', 'L195', 'L196', 'L197']\n",
 'lines': [{'lineID': 'L194',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.\n'},
  {'lineID': 'L195',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Well, I thought we'd start with pronunciation, if that's okay with you.\n"},
  {'lineID': 'L196',
   'characterID': 'u0',
   'movieID': 'm0',
   'character': 'BIANCA',
   'text': 'Not the hacking and gagging and spitting part.  Please.\n'},
  {'lineID': 'L197',
   'characterID': 'u2',
   'movieID': 'm0',
   'character': 'CAMERON',
   'text': "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?\n"}]}

In [7]:
extractSentencePairs(conversations)

[['Can we make this quick?  Roxanne Korrine and Andrew Barrett are having an incredibly horrendous public break- up on the quad.  Again.',
  "Well, I thought we'd start with pronunciation, if that's okay with you."],
 ["Well, I thought we'd start with pronunciation, if that's okay with you.",
  'Not the hacking and gagging and spitting part.  Please.'],
 ['Not the hacking and gagging and spitting part.  Please.',
  "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?"],
 ["You're asking me out.  That's so cute. What's your name again?",
  'Forget it.'],
 ["No, no, it's my fault -- we didn't have a proper introduction ---",
  'Cameron.'],
 ['Cameron.',
  "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does."],
 ["The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
  'Seems like she could get a date easy enough...'],
 [

# 데이터 로딩 및 손질

이제 단어장을 만들고 질의/응답문 쌍을 메모리에 적재하자.

우리는 수치공간에 embedding된 단어를 사용하는게 아니기 때문에, 데이터셋에서 만나는 각 단어에 인덱스를 지정해야 한다. 

이를 위해서 `Voc` 클래스를 만들어 단어와 인덱스 사이의 매핑 및 발단어별 등장 횟수를 저장하고자 한다. 이 클래스는 단어 하나를 더하거나(`addWord`), 문장 속 모든 단어를 더하거나(`addSentence`), 등장횟수가 적은 단어를 제거하는 메소드(`trim`)을 포함한다.

In [9]:
# 문장 토큰들
PAD_token = 0 # 짧은 문장에 패딩을 추가하는 데 사용
SOS_token = 1 # 문장 시작 토큰
EOS_token = 2 # 문장 종결 토큰

class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token:"PAD", SOS_token:"SOS", EOS_token:"EOS"}
        self.num_words = 3 # SOS, EOS, PAD 
        
    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1
    
    # 특정 등장횟수 이하의 단어는 삭제함
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        
        keep_words = []
        
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
                
        print('keep_words {} / {} = {:.4f}'.format(
            len(keep_words), len(self.word2index), len(keep_words) /
            len(self.word2index)
        ))
            
        # 딕셔너리 재설정
        self.word2index = {}
        self.word2count = {}
        self.index2word = {PAD_token: "PAD", SOS_token: "SOS", EOS_token: "EOS"}
        self.num_words = 3 # Count default tokens
        
        for word in keep_words:
            self.addWord(word)

이제 단어장과 질의/응답문 쌍을 처리하자. 이 데이터를 사용하기 전에 전처리가 필요하다.

먼저 `unicodeToAscii` 메소드를 사용하여 유니코드 문자열을 아스키로 변환해야한다. 다음으로 모든 문자를 소문자로 변환하고 기본적인 구두점들을 제외한 비문자들을 제거한다(`normalizeString`). 마지막으로 훈련이 수렴하는데 도움을 주기 위해 길이가 `MAX_LENGTH` 를 넘는 문자열을 제거한다(`filterPairs`).

In [17]:
MAX_LENGTH = 10 # 허용가능한 최대 단어수

# https://stackoverflow.com/a/518232/2809427 를 참고하여
# 유니코드 문자열을 아스키로 변환
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# 소문자 변환, 공백 제거, 비문자열 제거
def normalizeString(s):
    s = unicodeToAscii(s.lower())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+",r" ", s)
    s = re.sub(r"\s+", r" ", s).strip()
    return s

# 질의/응답쌍을 읽은 후 voc 객체 반환
def readVocs(datafile, corpus_name):
    print("Reading lines...")
    # 파일을 읽은 후 라인들로 분리
    lines = open(datafile, encoding='utf-8').\
        read().strip().split('\n')
    # 각 줄을 쌍으로 나눈 후 정규화
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]
    voc = Voc(corpus_name)
    return voc, pairs

# 주어진 문자열쌍 모두가 MAX_LENGTH 이하일 때만 True를 반환
def filterPair(p):
    # 입력 문자열은 EOS 토큰을 위해 마지막 문자를 보존해야함
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

# filterPair 조건을 사용하여 대화쌍을 필터링
def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

# 위에서 정의한 함수들을 이용하여 voc 객체와 쌍 리스트 반환
def loadPrepareData(corpus, corpus_name, datafile, save_dir):
    print("Start preparing training data...")
    voc, pairs = readVocs(datafile, corpus_name)
    print("Read {!s} sentence pairs".format(len(pairs)))
    pairs = filterPairs(pairs)
    print("Trimmed to {!s} sentence pairs".format(len(pairs)))
    print("Counting words...")
    for pair in pairs:
        voc.addSentence(pair[0])
        voc.addSentence(pair[1])
    print("Counted words:", voc.num_words)
    return voc, pairs


save_dir = os.path.join("data","save")
voc, pairs = loadPrepareData(corpus, corpus_name, datafile, save_dir)
print('\npairs:')
for pair in pairs[:10]:
    print(pair)

Start preparing training data...
Reading lines...
Read 221282 sentence pairs
Trimmed to 64271 sentence pairs
Counting words...
Counted words: 18008

pairs:
['there .', 'where ?']
['you have my word . as a gentleman', 'you re sweet .']
['hi .', 'looks like things worked out tonight huh ?']
['you know chastity ?', 'i believe we share an art instructor']
['have fun tonight ?', 'tons']
['well no . . .', 'then that s all you had to say .']
['then that s all you had to say .', 'but']
['but', 'you always been this selfish ?']
['do you listen to this crap ?', 'what crap ?']
['what good stuff ?', 'the real you .']


훈련 과정에서 빠른 수렴을 위한 또다른 전략은 단어장에서 드물게 등장하는 단어를 제거하는 것이다. 특성공간을 축소하는 것은 모델이 학습해야하는 어려움 또한 줄여준다. 우리는 다음 두 단계를 통해 이를 수행할 것이다.

1. `voc.trim` 함수를 이용하여 `MIN_COUNT` 이하의 단어를 제거
2. 제거된 단어가 포함된 문자쌍 제거

In [18]:
MIN_COUNT = 3 # 최소 등장수

def trimRareWords(voc, pairs, MIN_COUNT):
    voc.trim(MIN_COUNT)
    # 제거된 단어가 포함된 쌍 제거
    keep_pairs = []
    for pair in pairs:
        input_sentence = pair[0]
        output_sentence = pair[1]
        keep_input = True
        keep_output = True
        # 입력 문자열 체크
        for word in input_sentence.split(' '):
            if word not in voc.word2index:
                keep_input = False
                break
        # 출력 문자열 체크
        for word in output_sentence.split(' '):
            if word not in voc.word2index:
                keep_output = False
                break
                
        if keep_input and keep_output:
            keep_pairs.append(pair)
        
    print("Trimmed from {} pairs to {}, {:4f} of total".format(len(pairs),\
    len(keep_pairs), len(keep_pairs)/len(pairs)))
    return keep_pairs
    
pairs = trimRareWords(voc, pairs, MIN_COUNT)

keep_words 7823 / 18005 = 0.4345
Trimmed from 64271 pairs to 53165, 0.827200 of total


# 모델에 주입할 데이터 준비